In [1]:
versioninfo()

Julia Version 1.8.5
Commit 17cfb8e65ea (2023-01-08 06:45 UTC)
Platform Info:
  OS: Linux (x86_64-linux-gnu)
  CPU: 12 × Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-13.0.1 (ORCJIT, skylake)
  Threads: 1 on 12 virtual cores


## 8-4. 解析と評価

### 8-4-1. `Meta.parse()`

#### コード8-9. `Meta.parse()` の例(1)

In [2]:
src1 = "1 + 1"

"1 + 1"

In [3]:
ex1 = Meta.parse(src1)

:(1 + 1)

In [4]:
dump(ex1)

Expr
  head: Symbol call
  args: Array{Any}((3,))
    1: Symbol +
    2: Int64 1
    3: Int64 1


In [5]:
Meta.parse("p1 ∧ p2")  # `∧` は `\wedge`+TAB で入力できます

:(p1 ∧ p2)

In [6]:
dump(Meta.parse("p1 ∧ p2"))

Expr
  head: Symbol call
  args: Array{Any}((3,))
    1: Symbol ∧
    2: Symbol p1
    3: Symbol p2


In [7]:
Meta.parse("a")

:a

In [8]:
typeof(Meta.parse("a"))

Symbol

In [9]:
Meta.parse("\"文字列\"")

"文字列"

In [10]:
typeof(Meta.parse("\"文字列\""))

String

In [11]:
Meta.parse("(a + b)2")

LoadError: Base.Meta.ParseError("extra token \"2\" after end of expression")

In [12]:
(a + b)2

LoadError: syntax: extra token "2" after end of expression

#### コード8-10. `Meta.parse()` の例(2)

In [13]:
Meta.parse(join(["a$i" for i in 1:5], "+"))
# eqivalent to `Meta.parse("a1+a2+a3+a4+a5")`

:(a1 + a2 + a3 + a4 + a5)

In [14]:
Meta.parse(strip("#a + b#2", ['#', '2']))
# equivalent to `Meta.parse("a + b")`

:(a + b)

### 8-4-2. `eval()`

#### コード8-11. `eval()` の例(1)

In [15]:
ex1 = Meta.parse("1 + 1")  # `ex1 = :(1 + 1)` としても全く同じ

:(1 + 1)

In [16]:
eval(ex1)  # `1 + 1` の結果を返す

2

In [17]:
a = 97;
eval(:a)  # シンボルを渡すとその識別子が表す値を返す

97

In [18]:
ex2 = :(a + b)

:(a + b)

In [19]:
eval(ex2)  # `b` にまだ何も代入されていないのでエラーとなる

LoadError: UndefVarError: b not defined

In [20]:
b = 3;
eval(ex2)  # エラーにならず `a + b`（＝`97 + 3`）の結果を返す

100

#### コード8-12. `:(a + b - c)`

In [21]:
ex3 = :(a + b - c)

:((a + b) - c)

In [22]:
dump(ex3)

Expr
  head: Symbol call
  args: Array{Any}((3,))
    1: Symbol -
    2: Expr
      head: Symbol call
      args: Array{Any}((3,))
        1: Symbol +
        2: Symbol a
        3: Symbol b
    3: Symbol c


#### コード8-13. `eval(:(a + b - c))` の評価の流れ（エミュレーション）

In [23]:
a = 1; b = 3; c = 2;

In [24]:
ex3.args[1]

:-

In [25]:
eval(ex3.args[1])

- (generic function with 211 methods)

In [26]:
ex3.args[2]

:(a + b)

In [27]:
ex3.args[2].args[1]

:+

In [28]:
eval(ex3.args[2].args[1])

+ (generic function with 206 methods)

In [29]:
ex3.args[2].args[2]

:a

In [30]:
eval(ex3.args[2].args[2])

1

In [31]:
ex3.args[2].args[3]

:b

In [32]:
eval(ex3.args[2].args[3])

3

In [33]:
eval(ex3.args[2])  # compute `1 + 3`

4

In [34]:
ex3.args[3]

:c

In [35]:
eval(ex3.args[3])

2

In [36]:
eval(ex3)  # compute `4 - 2`

2

#### コード8-14. `ex4 = :(iseven(a) ? :even : :odd)` とその評価

In [37]:
ex4 = :(iseven(a) ? :even : :odd)

:(if iseven(a)
      :even
  else
      :odd
  end)

In [38]:
dump(ex4)

Expr
  head: Symbol if
  args: Array{Any}((3,))
    1: Expr
      head: Symbol call
      args: Array{Any}((2,))
        1: Symbol iseven
        2: Symbol a
    2: QuoteNode
      value: Symbol even
    3: QuoteNode
      value: Symbol odd


In [39]:
a = 1

1

In [40]:
ex4.head

:if

In [41]:
ex4.args[1]

:(iseven(a))

In [42]:
ex4.args[1].head

:call

In [43]:
ex4.args[1].args[1]

:iseven

In [44]:
eval(ex4.args[1].args[1])

iseven (generic function with 5 methods)

In [45]:
ex4.args[1].args[2]

:a

In [46]:
eval(ex4.args[1].args[2])

1

In [47]:
eval(ex4.args[1])

false

In [48]:
ex4.args[3]

:(:odd)

In [49]:
eval(ex4.args[3])

:odd

In [50]:
eval(ex4)

:odd

### 8-4-3. 式展開

#### コード8-15. `Expr` の式展開の例(1)

In [51]:
a = 97;
ex3 = :($a + b)

:(97 + b)

In [52]:
b = 3;
eval(ex3)  # `97 + b` が評価される

100

In [53]:
a = 0; b = 1;  # ここで `a` に別の値を代入しても `ex3` に影響はない
eval(ex3)  #  `97 + b` が評価される

98

In [54]:
ex3a = :($(a + b) + c)  # `a == 0`, `b == 1` なので `a + b == 1`

:(1 + c)

In [55]:
c = 10;
eval(ex3a)

11

#### コード8-16. `Expr` の式展開の例(2)

In [56]:
args = (n^2 for n=1:5);

In [57]:
ex4_fn = :(fn($(args...)))

:(fn(1, 4, 9, 16, 25))

In [58]:
ex4_fn2 = :(fn(0, $(args...)))

:(fn(0, 1, 4, 9, 16, 25))

In [59]:
ex4_vector = :([$(args...)])

:([1, 4, 9, 16, 25])

In [60]:
ex4_vector = :([0, $(args...)])

:([0, 1, 4, 9, 16, 25])

In [61]:
ex4_tuple = :(($(args...),))

:((1, 4, 9, 16, 25))

In [62]:
ex4_tuple = :((0, $(args...)))

:((0, 1, 4, 9, 16, 25))

In [63]:
"$(args...)"  # 参考（比較）

"1491625"

#### コード8−17. `Expr` の式展開の例(3)

In [64]:
qx = :x;
qy = :y;
ex5_add = :($qx + $qy)

:(x + y)

In [65]:
x = 3;
y = 4;
eval(ex5_add)

7

In [66]:
args = (:x, :y, 5);
ex5_muladd = :(muladd($(args...)))

:(muladd(x, y, 5))

In [67]:
eval(ex5_muladd)

17

In [68]:
qs_even = QuoteNode(:even);
qs_odd = QuoteNode(:odd);
quote
    if iseven(a)
        $qs_even
    else
        $qs_odd
    end
end |> Base.remove_linenums!

quote
    if iseven(a)
        :even
    else
        :odd
    end
end

#### コード8-18. `Expr` の式展開の例(4)

In [69]:
dump(:(1//2 + 1))

Expr
  head: Symbol call
  args: Array{Any}((3,))
    1: Symbol +
    2: Expr
      head: Symbol call
      args: Array{Any}((3,))
        1: Symbol //
        2: Int64 1
        3: Int64 2
    3: Int64 1


In [70]:
dump(:($(1//2) + 1))  # `1//2` という「値」をアンクォート

Expr
  head: Symbol call
  args: Array{Any}((3,))
    1: Symbol +
    2: Rational{Int64}
      num: Int64 1
      den: Int64 2
    3: Int64 1


In [71]:
dump(:($(:(1//2)) + 1))  # `:(1//2)` という `Expr` をアンクォート

Expr
  head: Symbol call
  args: Array{Any}((3,))
    1: Symbol +
    2: Expr
      head: Symbol call
      args: Array{Any}((3,))
        1: Symbol //
        2: Int64 1
        3: Int64 2
    3: Int64 1


In [72]:
:($(:(1//2)) + 1) == :(1//2 + 1)

true

### 8-4-4. コードの動的生成

#### コード8-19. `OSType` の定義(1): 基本実装

In [73]:
abstract type OSType end

In [74]:
OSType(osname) = OSType(Val(Symbol(osname)))

OSType

In [75]:
struct OtherOS <: OSType
    name::Symbol
end

In [76]:
OSType(::Val{other}) where {other} = OtherOS(other)

OSType

In [77]:
Base.print(io::IO, other::OtherOS) = print(io, other.name)

#### 仮想コード8-1. `OSType` の定義(2): `Linux` 型の追加

```julia
struct Linux <: OSType end

OSType(::Val{:Linux}) = Linux()

Base.print(io::IO, ::Linux) = print(io, "Linux")
```

#### コード8-20. `OSType` の定義(3): 複数の型の一括定義

```julia
for osname in ["Linux", "Mac", "Windows"]
    typename = Symbol(osname)
    eval(:(struct $typename <: OSType end))
    eval(:(OSType(::Val{$(QuoteNode(typename))}) = $typename()))
    eval(:(Base.print(io::IO, ::$typename) = print(io, $osname)))
end
```

#### コード8-21. `OSType` の定義(4): 複数の型の一括定義(改)

In [78]:
for osname in ["Linux", "Mac", "Windows"]
    typename = Symbol(osname)
    @eval struct $typename <: OSType end
    @eval OSType(::Val{$(QuoteNode(typename))}) = $typename()
    @eval Base.print(io::IO, ::$typename) = print(io, $osname)
end

#### コード8-22. `OSType` の動作確認

In [79]:
subtypes(OSType)

4-element Vector{Any}:
 Linux
 Mac
 OtherOS
 Windows

In [80]:
OSType.([:Linux, :Mac, :Windows, :FreeBSD])

4-element Vector{OSType}:
 Linux()
 Mac()
 Windows()
 OtherOS(:FreeBSD)

In [81]:
OSType.([:Linux, :Mac, :Windows, :FreeBSD]) .|> string

4-element Vector{String}:
 "Linux"
 "Mac"
 "Windows"
 "FreeBSD"